In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import matplotlib.animation as animation
import matplotlib.patches as patches
import matplotlib.transforms as mplTransforms
from matplotlib.path import Path

In [10]:
"""
Useful functions for reflection

"""
def find_nearest_normal(point,path):
    ## Find the normal vector to a path that is closest to a point
    num_vertices = np.size(path.vertices[0:-1,0])
    distances = np.zeros((num_vertices))
    for i in range(num_vertices):
        ## For every line segment, find the distance
        line_segment = path.vertices[i+1]-path.vertices[i]
        line_to_point = path.vertices[i+1]-point
        distances[i] = np.linalg.norm(np.cross(line_to_point,line_segment))/np.linalg.norm(line_segment)
    ## The closest line has the least distance. Find the starting vertex and the line segment
    nearest_line_start_vertex = path.vertices[np.argmin(distances)]
    line_vector = path.vertices[np.argmin(distances)+1] - path.vertices[np.argmin(distances)] 
    nearest_normal = np.array([[0,-1],[1,0]]).dot(line_vector) ## The normal vector is [-b,a]
    return nearest_normal/np.linalg.norm(nearest_normal)  ## Make it a unit vector

"""

Class definition for the box.

    Methods:
        - box = Box(500): Initialize a 500 particle box
        - box.animate(): video of the atoms flying in the box
        - box.momentum_focus(): simulate the momentum focusing observation
        
"""


class Box():
    
    def __init__(self, NumParticles=500):
        ## Initial properties
        self.omega = 1.25
        self.dt = 0.01
        self.k_F = 2
        self.NumParticles = NumParticles
        
        ## Box shape - shear tilted
        verts = [
            (-0.5, -1.5), # left, bottom
            (-1., 1.5), # left, top
            (0.5, 1.5), # right, top
            (1., -1.5), # right, bottom
            (-0.5, -1.5), # ignored
                ]

        codes = [Path.MOVETO,
                 Path.LINETO,
                 Path.LINETO,
                 Path.LINETO,
                 Path.CLOSEPOLY,
                 ]

        self.box_path = Path(verts, codes)
        
        ## Initial positions
        self.positions = [1,0.5]*(np.random.rand(NumParticles,2)-0.5)
        
        ## Momentum distribution (Assuming T=0)
        r_points = 2*np.random.rand(NumParticles,1) 
        theta_points = 2*math.pi*np.random.rand(NumParticles,1)
        self.velocities = np.zeros((NumParticles,2))
        self.velocities = np.hstack((np.sqrt(r_points)*np.cos(theta_points),
                               np.sqrt(r_points)*np.sin(theta_points)))

        
    def update_positions(self):
        
        ## Update the box forward in time to t+dt
        step = self.velocities*self.dt
        positions_test = self.positions + step
          
        ## If a particle hits a wall, reflect it
        for j in range(np.size(self.positions[:,1])):
            ## Take every particle, and check if it's in the box_path
            if not(self.box_path.contains_point(positions_test[j,:])):
                ## Find the  outward-pointing normal vector
                normal = find_nearest_normal(positions_test[j,:],self.box_path)
                ## Reflect about normal vector
                self.velocities[j,:] = self.velocities[j,:] - 2*(normal.dot(self.velocities[j,:]))*normal

        ## Advance
        step = self.velocities*self.dt
        self.positions = self.positions + step
        self.velocities[:,1] = self.velocities[:,1] - self.omega**2 * self.positions[:,1] * self.dt

    def update_plot(self,t,box,scat):
        ## Update the plot for the animation method below
        self.update_positions()
        scat.set_offsets(self.positions)
        return scat

    def animate(self):
        ## Animate the box continuously
        fig = plt.figure()
        ax = fig.add_axes([0, 0, 1, 1])
        ax.set_xlim(-2, 2), ax.set_xticks([])
        ax.set_ylim(-2, 2), ax.set_yticks([])
        ## Draw the box
        patch = patches.PathPatch(self.box_path, fill=False, lw=2)
        ax.add_patch(patch)
        ## Draw everything else
        scat = ax.scatter(self.positions[:,0],self.positions[:,1],
                          s=30, alpha=0.5, color ='r')
        ani = animation.FuncAnimation(fig, box.update_plot, fargs=(self,scat), 
                                      interval=5, repeat=True)
        plt.show()


    def display(self):
        ## Display the current state of the box
        fig = plt.figure()
        ax = fig.add_axes([0, 0, 1, 1])
        ax.set_xlim(-2, 2), ax.set_xticks([])
        ax.set_ylim(-2, 2), ax.set_yticks([])
        ## Draw the box
        patch = patches.PathPatch(self.box_path, fill=False, lw=2)
        ax.add_patch(patch)
        ## Draw everything else
        scat = ax.scatter(self.positions[:,0],self.positions[:,1],
                          s=30, alpha=0.5, color ='r')
        plt.show()
        
    def momentum_focus(self):
        ## Momentum focus the box
        NumFrames = math.floor(math.pi/(self.dt*2*self.omega))
        for i in range(NumFrames):
            self.update_positions()
            
        ## Process Data
#         hist, bin_edges = np.histogram(self.positions[:,1])
#         plt.plot(bin_edges[0:-1],hist)
        plt.hist(self.positions[:,1],30)
        plt.show()


    

In [12]:
box = Box(500)
box.animate()